Mooncoin Market Analysis
========================

Documentation:
- https://github.com/ccxt/ccxt/wiki

In [1]:
import ccxt
from pydash import py_ as _
from datetime import datetime, timedelta
from dateutil.parser import parse
import pandas as pd
import datetime
import numpy as np
import matplotlib
import re

pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 8)
pd.set_option('display.max_rows', 8)

def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')

In [2]:
bleutrade    = ccxt.bleutrade({"timeout": 100*1000})
bittrex      = ccxt.bittrex()

In [3]:
moonbtc_trades = pd.DataFrame(bleutrade.fetch_trades("MOON/BTC", params={"count": 200}), columns=['amount', 'datetime', 'side', 'info'])
moonbtc_trades['price']      = moonbtc_trades['info'].map(lambda x: float(x['Price']))
moonbtc_trades['btc_amount'] = moonbtc_trades['amount'] * moonbtc_trades['price']
del moonbtc_trades['info']
moonbtc_trades['datetime'] = pd.to_datetime(moonbtc_trades['datetime']) # convert seconds to nanoseconds
moonbtc_trades

           amount            datetime  side         price  btc_amount
0    8.685796e+05 2018-01-02 22:06:16  sell  2.000000e-08    0.017372
1    1.214861e+06 2018-01-02 21:55:30  sell  2.000000e-08    0.024297
2    1.500000e+07 2018-01-02 21:55:30  sell  2.000000e-08    0.300000
3    5.000000e+04 2018-01-02 21:55:30  sell  2.000000e-08    0.001000
..            ...                 ...   ...           ...         ...
196  2.195130e+05 2018-01-02 16:37:27  sell  2.000000e-08    0.004390
197  1.245472e+06 2018-01-02 16:33:30   buy  3.000000e-08    0.037364
198  1.254528e+06 2018-01-02 16:33:30   buy  3.000000e-08    0.037636
199  4.028639e+06 2018-01-02 16:24:36   buy  3.000000e-08    0.120859

[200 rows x 5 columns]

Mooncoin Volumes
----------------

**How far back in time does bleutrade order moonbtc_trades go?**

- 2017-12-31: for max moonbtc_trades of 200 orders, Timedelta('0 days 11:28:11'), which is less than a day

In [4]:
moonbtc_trades['datetime'].max() - moonbtc_trades['datetime'].min()

Timedelta('0 days 05:41:40')

**What is the average order size?**

- 2017-12-31: Average orders are BTC 0.006385 (buy@2) - 0.007357 (sell@1) which is about $85-100 USD
- 2017-12-31: Average orders are BTC 0.034786 (buy@2) - 0.000630 (sell@1) which is an order of magnitude shift in both directions

In [5]:
moonbtc_trades.groupby('side').mean()

             amount         price  btc_amount
side                                         
buy   782626.205834  3.000000e-08    0.023479
sell  601019.583023  2.000000e-08    0.012020

**What is the relative volume on the buy and sell sides of the market**

- 2017-12-31: There is currently 2.9x BTC / 5.8x MOON volume on the sell@1 side of the market
- 2018-01-01: There is currently 3.4 BTC / 6.8x MOON volume on the buy@2 side of the market

In [6]:
moonbtc_trades_sum = moonbtc_trades[['amount', 'btc_amount','side']].groupby('side').sum()
moonbtc_trades_sum

            amount  btc_amount
side                          
buy   6.261010e+07    1.878303
sell  7.212235e+07    1.442447

In [7]:
moonbtc_trades_sum.loc['sell'] / moonbtc_trades_sum.loc['buy']

amount        1.151928
btc_amount    0.767952
dtype: float64

**Can we plot a chart for the buy/sell volume history**

- 2017-12-31 - Apart from a massive 0.9 BTC sell@1 spike at 7am, volumes remain low but about even on both sides of the market
- 2018-01-01 - Massive 5 BTC buy@2 spike at 10am

In [8]:
moonbtc_trades_agg = moonbtc_trades.groupby([moonbtc_trades['datetime'].dt.hour, 'side'])['btc_amount'].sum()
moonbtc_trades_agg.unstack().plot()
moonbtc_trades_agg

datetime  side
16        buy     0.417574
          sell    0.094426
17        buy     0.562984
          sell    0.000180
                    ...   
20        sell    0.211249
21        buy     0.306285
          sell    0.922017
22        sell    0.017372
Name: btc_amount, Length: 13, dtype: float64

Price
-----

** What is the price of mooncoin **

In [9]:
tickers = {
    'bleutrade': bleutrade.fetch_tickers(),
    'bittrex':   bittrex.fetch_tickers(),
}
tickers['bleutrade']['MOON/BTC']

{'ask': 3e-08,
 'average': None,
 'baseVolume': 1676188790.1925051,
 'bid': 2e-08,
 'change': None,
 'close': None,
 'datetime': '2018-01-02T22:09:27.000Z',
 'first': None,
 'high': 4e-08,
 'info': {'Ask': '0.00000003',
  'Average': '0.00000003',
  'BaseCurrency': 'Bitcoin',
  'BaseVolume': '48.34729255',
  'Bid': '0.00000002',
  'High': '0.00000004',
  'IsActive': 'true',
  'Last': '0.00000002',
  'Low': '0.00000002',
  'MarketCurrency': 'Mooncoin',
  'MarketName': 'MOON_BTC',
  'PrevDay': '0.00000003',
  'TimeStamp': '2018-01-02 22:09:27',
  'Volume': '1676188790.19250512'},
 'last': 2e-08,
 'low': 2e-08,
 'open': None,
 'percentage': None,
 'quoteVolume': 48.34729255,
 'symbol': 'MOON/BTC',
 'timestamp': 1514930967000,
 'vwap': None}

In [10]:
def bid(market):       return market['bid']
def mid(market):       return (market['ask'] + market['bid'])/2
def spread(market):    return (market['ask'] - market['bid'])
def spread_pc(market): return spread(market)/mid(market)

In [11]:

price = {'bittrex': {}, 'bleutrade': {}}
price['bittrex']['BTC/USDT']    = bid(tickers['bittrex']['BTC/USDT'])
price['bittrex']['DOGE/BTC']    = bid(tickers['bittrex']['DOGE/BTC'])
price['bittrex']['ETH/BTC']     = bid(tickers['bittrex']['ETH/BTC'])
price['bleutrade']['DOGE/BTC']  = bid(tickers['bleutrade']['DOGE/BTC'])
price['bleutrade']['ETH/BTC']   = bid(tickers['bleutrade']['ETH/BTC'])
price['bleutrade']['MOON/BTC']  = bid(tickers['bleutrade']['MOON/BTC'])
price['bleutrade']['MOON/DOGE'] = bid(tickers['bleutrade']['MOON/DOGE'])
price['bleutrade']['MOON/ETH']  = bid(tickers['bleutrade']['MOON/ETH']) 

price['bleutrade']['MOON/ETH/BTC']     = price['bleutrade']['MOON/ETH']  * price['bleutrade']['ETH/BTC']
price['bleutrade']['MOON/DOGE/BTC']    = price['bleutrade']['MOON/DOGE'] * price['bleutrade']['DOGE/BTC']
price['bittrex']['MOON/ETH/BTC']       = price['bleutrade']['MOON/ETH']  * price['bittrex']['ETH/BTC']
price['bittrex']['MOON/DOGE/BTC']      = price['bleutrade']['MOON/DOGE'] * price['bittrex']['DOGE/BTC']
price['bittrex']['MOON/ETH/BTC/USDT']  = price['bleutrade']['MOON/ETH/BTC']  * price['bittrex']['BTC/USDT']
price['bittrex']['MOON/DOGE/BTC/USDT'] = price['bleutrade']['MOON/DOGE/BTC'] * price['bittrex']['BTC/USDT']
price['bittrex']['MOON/BTC/USDT']      = price['bleutrade']['MOON/BTC']      * price['bittrex']['BTC/USDT']                                                                                         

price                                                                                              

{'bittrex': {'BTC/USDT': 14940.00000001,
  'DOGE/BTC': 6.1e-07,
  'ETH/BTC': 0.05719307,
  'MOON/BTC/USDT': 0.00029880000000020003,
  'MOON/DOGE/BTC': 2.09931439e-08,
  'MOON/DOGE/BTC/USDT': 0.000303354370854203,
  'MOON/ETH/BTC': 2.17333666e-08,
  'MOON/ETH/BTC/USDT': 0.00032855370022821995},
 'bleutrade': {'DOGE/BTC': 5.9e-07,
  'ETH/BTC': 0.05787249,
  'MOON/BTC': 2e-08,
  'MOON/DOGE': 0.03441499,
  'MOON/DOGE/BTC': 2.0304844099999997e-08,
  'MOON/ETH': 3.8e-07,
  'MOON/ETH/BTC': 2.19915462e-08}}

In [12]:
bleutrade_markets = pd.DataFrame(bleutrade.fetch_tickers()).transpose()[['ask', 'bid', 'last']]
bleutrade_markets['mid']    = (bleutrade_markets['ask'] + bleutrade_markets['bid']) / 2
bleutrade_markets['spread'] = (bleutrade_markets['ask'] - bleutrade_markets['bid']) / bleutrade_markets['bid']
bleutrade_markets = bleutrade_markets.filter(regex="MOON/|ETH/|DOGE/", axis=0)
bleutrade_markets

                 ask        bid       last         mid      spread
DOGE/BTC       6e-07    5.9e-07      6e-07    5.95e-07   0.0169492
DOGE/ETH   1.212e-05  1.015e-05  1.015e-05  1.1135e-05    0.194089
ETH/BTC    0.0578725  0.0574794  0.0578725    0.057676  0.00683862
ETH/DOGE      108549    92685.2    91082.8      100617    0.171155
MOON/BTC       3e-08      2e-08      2e-08     2.5e-08         0.5
MOON/DOGE   0.038415   0.034415   0.038415    0.036415    0.116229
MOON/ETH     3.9e-07    3.8e-07    3.8e-07    3.85e-07   0.0263158

In [13]:
for symbol in bleutrade_markets.transpose():
    top  = re.split('/', symbol)[0]
    base = re.split('/', symbol)[-1]
    if base == 'BTC': continue
    if base in ['ETH', 'DOGE']:
        bleutrade_markets.loc[symbol+'/BTC'] = bleutrade_markets.loc[symbol] * bleutrade_markets.loc[base+'/BTC'] 
        bleutrade_markets.loc[symbol+'/BTC']['spread'] = bleutrade_markets.loc[symbol]['spread'] + bleutrade_markets.loc[base+'/BTC']['spread']         

print_full(bleutrade_markets)

                       ask          bid         last          mid      spread
DOGE/BTC             6e-07      5.9e-07        6e-07     5.95e-07   0.0169492
DOGE/ETH         1.212e-05    1.015e-05    1.015e-05   1.1135e-05    0.194089
ETH/BTC          0.0578725    0.0574794    0.0578725     0.057676  0.00683862
ETH/DOGE            108549      92685.2      91082.8       100617    0.171155
MOON/BTC             3e-08        2e-08        2e-08      2.5e-08         0.5
MOON/DOGE         0.038415     0.034415     0.038415     0.036415    0.116229
MOON/ETH           3.9e-07      3.8e-07      3.8e-07     3.85e-07   0.0263158
DOGE/ETH/BTC   7.01415e-07  5.83416e-07  5.87406e-07  6.42222e-07    0.200927
ETH/DOGE/BTC     0.0651292    0.0546843    0.0546497    0.0598671    0.188104
MOON/DOGE/BTC   2.3049e-08  2.03048e-08   2.3049e-08  2.16669e-08    0.133178
MOON/ETH/BTC   2.25703e-08  2.18422e-08  2.19915e-08  2.22052e-08   0.0331544


All MoonCoin Markets
--------------------

In [14]:
tickers = {}
moon_exchanges = 'bleutrade'
for exchange_name in list(moon_exchanges or ccxt.exchanges):
    try: 
        ticker  = getattr(ccxt, exchange_name)().fetch_tickers()
        symbols = list(filter(lambda symbol: "MOON" in symbol, ticker.keys()))
        if len(symbols):            
            if not exchange_name in moon_exchanges: moon_exchanges += [exchange_name]
            for symbol in symbols:
                ticker[symbol]['exchange'] = exchange_name
                tickers[exchange_name+':'+symbol] = ticker[symbol]
    except: pass
        
moon_exchanges

'bleutrade'

In [15]:
tickers_df = pd.DataFrame(tickers)
tickers_df.transpose()

Empty DataFrame
Columns: []
Index: []